In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.data import Dataset, AUTOTUNE
from tensorflow import keras
import re
from keras import losses

2024-10-03 21:30:11.113898: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-03 21:30:11.121283: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-03 21:30:11.130000: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-03 21:30:11.132942: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-03 21:30:11.140343: I tensorflow/core/platform/cpu_feature_guar

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


I0000 00:00:1727980212.106306    7795 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727980212.145805    7795 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727980212.145838    7795 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
raw = open('data2.txt', mode='r', encoding='utf-8').readlines()
data = []
for line in raw:
    if line != '\n' and 'Глава' not in line:
        data.append(' '.join(line.split()[1:]))
    
data = [line.replace('\n', ' ').replace('\xa0', ' ') for line in data]
text = ' '.join(data)

In [4]:
text[:100]

'завет ЗАВЕТ Моисея  В начале сотворил Бог небо и землю. Земля же была безвидна и пуста, и тьма над б'

In [5]:
def get_features_target(seq):
    features = seq[:-1]
    target = seq[1:]
    return features, target

BATCH_SIZE = 32

word_pattern = r'[^а-яА-ЯёЁ0-9 :,-]'
clear_text = re.sub(word_pattern, '', text)

alphabet = np.array(sorted(set(clear_text.split(' '))))

clear_text = np.array(clear_text.split(' '))
clear_text = clear_text[clear_text != '']

word_to_idx = {}
idx_to_word = {}

for idx, word in enumerate(alphabet):
    word_to_idx[word] = idx
    idx_to_word[idx] = word

text_idx = np.array([word_to_idx[word] for word in clear_text])
sequences = Dataset.from_tensor_slices(text_idx).batch(BATCH_SIZE, drop_remainder=True)
dataset = sequences.map(get_features_target)

data = dataset.batch(BATCH_SIZE, drop_remainder=True).repeat()
data = data.prefetch(AUTOTUNE)

I0000 00:00:1727980212.615193    7795 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727980212.615232    7795 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727980212.615242    7795 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727980212.684239    7795 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727980212.684283    7795 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-03

In [6]:
model = keras.Sequential([
    keras.layers.Embedding(len(alphabet), 64),
    keras.layers.LSTM(64, return_sequences=True, stateful=True),
    keras.layers.Dense(len(alphabet), activation='relu'),
])

model.compile(optimizer='adam', loss=losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.fit(data, epochs=20, verbose=1, steps_per_epoch= len(sequences) // BATCH_SIZE)

Epoch 1/20
  1/651 ━━━━━━━━━━━━━━━━━━━━ 13:50 1s/step - accuracy: 0.0000e+00 - loss: 11.1204

2024-10-03 21:30:14.152334: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


651/651 ━━━━━━━━━━━━━━━━━━━━ 29s 43ms/step - accuracy: 0.0720 - loss: 9.7398
Epoch 2/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 28s 43ms/step - accuracy: 0.0748 - loss: 9.0568
Epoch 3/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 28s 43ms/step - accuracy: 0.0779 - loss: 8.9917
Epoch 4/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - accuracy: 0.0826 - loss: 8.9381
Epoch 5/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - accuracy: 0.0876 - loss: 8.8802
Epoch 6/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - accuracy: 0.0934 - loss: 8.8191
Epoch 7/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - accuracy: 0.0997 - loss: 8.7643
Epoch 8/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - accuracy: 0.1038 - loss: 8.7204
Epoch 9/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - accuracy: 0.1072 - loss: 8.6860
Epoch 10/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - accuracy: 0.1111 - loss: 8.6544
Epoch 11/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - accuracy: 0.1159 - loss: 8.6232
Epoch 12/20
651/651 ━━━━━━━━━━━━━━━━━━━━ 

In [7]:
def predict_next(sample, model, tokenizer, vocabulary, n_next, rnd_power, batch_size):
    sample_token = [tokenizer[char] for char in sample.split()]
    predicted = sample_token

    sample_tensor = tf.expand_dims(sample_token, 0)
    sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    
    for _ in range(n_next):
        cur = model(sample_tensor)
        cur = cur[0].numpy() / rnd_power
        cur = tf.random.categorical(cur, num_samples=1)[-1, 0].numpy()
        predicted.append(cur)
        sample_tensor = predicted[-99:]
        sample_tensor = tf.expand_dims([cur], 0)
        sample_tensor = tf.repeat(sample_tensor, batch_size, axis=0)
    res = [vocabulary[i] for i in predicted]
    generated = ' '.join(res)
    return generated

In [8]:
print(predict_next(
    sample='бог',
    model=model,
    tokenizer=word_to_idx,
    vocabulary=idx_to_word,
    n_next=20,
    rnd_power=0.6,
    batch_size=BATCH_SIZE,
))

бог не и на месте его и не было у тебя и не бы у нас А это не только в


In [9]:
print(predict_next(
    sample='1',
    model=model,
    tokenizer=word_to_idx,
    vocabulary=idx_to_word,
    n_next=20,
    rnd_power=0.6,
    batch_size=BATCH_SIZE,
))

1 сын его на землю и не с зависящими от Господа на нас и не будет с владением, и на земле
